## 命令行参数

- 脚本启动时命令行参数，以字符串形式保存在Python的`sys.argv`中<p>       
- 从Shell命令行启动脚本：`python test.py spam`，此时`sys.argv`为`['test.py','spam']`；交互命令行启动的Python没有命令行参数传入，`sys.argv`为空 <p>            
- 按惯例，可以将命令行参数设置成`-name value`的形式；用来传递要处理的文件名，处理模式的`flag`、`Internet`地址等     <p>   
- 对于复杂的命令行参数<span class="burk">，可以利用Python标准库`getopt`,`optparse`模块处理</span><p>  

In [ ]:
# %load scripts\testargv2.py
#!python3
# testargv2.py - 将传入的命令行参数保存在词典中


def getopts(argv):
    opts = {}
    while argv:
        if argv[0][0] == '-':
            opts[argv[0]] = argv[1]
            argv = argv[2:]
        else:
            argv = argv[1:]
    return opts


if __name__ == '__main__':
    from sys import argv
    myargvs = getopts(argv)
    if '-i' in myargvs:
        print(myargvs['-i'])
    print(myargvs)



## Shell 环境变量

- shell 环境变量通过`os.environ`访问，独立于Python系统，为程序提供系统级的配置<p>
- 访问环境变量，`os.environ['PATH']`；通过赋值改变环境变量，Python 退出后，变量会变回初始值

## 标准流

- 标准输入、输出和错误流，通用的程序通信方式；为预先打开的Python文件对象，在Python启动时被绑定到控制台窗口上 <p>  
    
- 内部的print和input函数只是标准输出\输入流的接口，默认情况下，标准输出会打印到程序启动的Console上，标准输入的文本从键盘读取 <p> 
    
- 命令行运行时，利用shell语法 `<filename` 将标准输入流重定向到文件输入，脚本从文件中而非键盘中读入内容； `>filename` 将标准输出流重定向到文件中；shell字符 `“|”`，将一个程序的标准输出发送到另一个程序的标准输入
> - `python test.py <input.text >output.text`
> - `python test.py <input.text | python reader.py`
> - `reader.py`脚本中的`input()`语句,只能接受`test.py`输出语句中的一行；前者中的`sys.stdin.readlines()`接受否则输出的所有语句为一个列表
> - 当`stdin`被重定向到文件或管道时，无法再获取键盘输入；可以利用`msvcrt`来实现
- 在Python内，<span class="mark">任何提供了类似文件`read`方法的对象可以指定给`sys.stdin`，</span>从该对象的`read`方法获取输入；<span class="mark">任何定义了类似文件`write`方法的对象可以指定给`sys.stdout`，</span>标准输出发送到该对象方法上。
> - `print('Hello')`语句，默认`end='\n'`,等价于`sys.stdout.write('Hello\n')`
> - 多个`print`语句，如`print('Hello'),print('Ok'),print('Bye')`等价于`sys.stdout.writelines(['Hello\n','Ok\n','Bye\n'])`
    
- `io.StringIO`和`io.BytesIO`对象实例可被指定给`sys.stdin`和`sys.stdout`以重定向`input`和`print`调用
> 重定向后，调用`print`屏幕不会显示，`print`的内容会保存在对象实例中
    
- `print(stuff,file=filename)`,将stuff打印到文件filename中<p>
- <span class="mark">使用场景：重定向可以为一个用来和命令行打交道的程序增加GUI接口</span>

In [1]:
import sys
for f in (sys.stdin,sys.stdout,sys.stderr):print(f)

<_io.TextIOWrapper name='<stdin>' mode='r' encoding='936'>


In [ ]:
# %load scripts\moreplus.py
import sys


def getreply():
    if sys.stdin.isatty():  # 如果 stdin 是控制台，从stdin读取数据
        return input('?:')
    else:
        if sys.platform[:3] == 'win':
            import msvcrt
            msvcrt.putch('?')
            key = msvcrt.getche()   # 获取键盘输入
            msvcrt.putch('\n')
            return key
        else:
            assert False, 'platform not supported'


def more(text, numlines=15):
    lines = text.splitlines()
    while lines:
        chunk = lines[:numlines]
        lines = lines[numlines:]
        for line in chunk:
            print(line)
        if lines and getreply() not in ['y', 'Y']:
            break

if __name__ == '__main__':
    if len(sys.argv) == 1:
        more(sys.stdin.read())
    else:
        more(open(sys.argv[1], 'rb').read())




In [22]:
from io import StringIO
import sys

buff=StringIO()
temp=sys.stdout

sys.stdout=buff
print(42,'spam',3.1425)

sys.stdout=temp
buff.getvalue()


'42 spam 3.1425\n'

In [1]:
import sys
